# Project

In [1]:
import sys

In [2]:
import re
_r = re.escape
def _re_replace(s : str, to_replace : dict):
    for p, r in to_replace.items():
        s = re.compile(p).sub(r, s)
    return s

In [3]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

In [4]:
!{sys.executable} -m pip install --quiet pandas
import pandas as pd

In [5]:
!{sys.executable} -m pip install --quiet seaborn
import seaborn as sns

In [6]:
# OLS using statsmodels
!{sys.executable} -m pip install --quiet statsmodels numpy
import statsmodels.api as sm
import numpy as np

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [1]:
!{sys.executable} -m pip install --quiet sklearn
import sklearn as skl

/bin/bash: {sys.executable}: command not found


## Data

The dataset<sup><a name="note-cariboo" href="#cariboo">[7]</a></sup> is composed of 7 different subsets, we will be using 4 of the datasets. Since each feature resides in different sets. 

- `Appearances.csv`
    - Player ID, Game ID, Appearance ID, Competition ID, Player club ID, Assist, Minutes Played, Yellow cards, Red Cards
- `Clubs.csv `
    - Club ID, Name, Pretty_name, Domestic_competition_id, Total_market_value, Squad_size, Average_age, Foreigners_numbers, Foreigners_percentage, National_team_players, Stadium_name, Stadium_seats, Net_transfer_record, Coach_name, URL
- `Competitions.csv`
    - Competition_id, Name, type, country_id, country_name, domestic_league_code, confederation, URL.  
- `Games.csv` 
    - Game_id, Competition_code, Season, Round, Date, Home_club_id, Away_club_id, Home_club_goals, away_club_goals, Home_club_postions, Away_club_postion, Stadium, Attendance, Referee, URL 
- `Leagues.csv` 
    - League_id, name, Confederation
- `Player_valuations.csv` 
    - Player_id, Date, Market_value
- `Players.csv`
    - Player_id, Last_season, Current_club_id, Name, Pretty_name, country_of_birth, Country_of_citizenship, Date_of_birth, Position, Sub_position, Foot, Height_in_cm, Market_value_in_gbp, Highest_market_value_in_gbp, URL

- _What an observation consists of_: We are trying to use the variables we assume to be the most important and independent from each other. We decided on 
    - Club, Nationality, League, Appearances, Minutes, Goals, Assist, Age, Conduct, Media Presence, Years Played, Trophies, Experience, Position, Physicality, Potential. 
- _What some critical variables are, how they are represented_: We want variables which have the highest co-variance with each other. The metric should handle most features as unique features. 
- _Any special handling, transformations, cleaning, etc will be needed_: There will be club names, and probably inferences in our data. Such as Media Presence or Potential, these are metrics which can be objective to the person. How popular is the player that we are analyzing? 

We are still going to be in search of more databases that might have different descriptive data that we might like to see how organizations search for talent. We can use what they might describe as their most sought out characteristics. 

For Simplicity we can also assume that all players have no contracts for their evaluation and are based solely on performance and the other variables mentioned.

In [8]:
_data_ = {
    name: pd.read_csv(
        file, 
        engine = 'c',
        low_memory = True,
        memory_map = False, # set `False` to load into memory
        **kwargs
    ) for name, file, kwargs in [
        ('appearances', 'data/appearances.csv', {
            'dtype': {
                'player_id': 'object',
                'game_id': 'object',
                'appearance_id': 'object',
                'competition_id': 'object',
                'player_club_id': 'object'
            }
        }),
        ('clubs', 'data/clubs.csv', {
            'dtype': {
                'club_id': 'object'
            }
        }),
        #('competitions', 'data/competitions.csv', {}),
        ('games', 'data/games.csv', {
            'dtype': {
                'game_id': 'object'
            }
        }),
        #('leagues', 'data/leagues.csv', {}),
        ('players', 'data/players.csv', {
            'parse_dates': ['date_of_birth'],
            'dtype': {
                'player_id': 'object',
                'country_of_birth': 'category',
                'country_of_citizenship': 'category',
                'position': 'category',
                'sub_position': 'category'
            }
        }),
        ('player_valuations', 'data/player_valuations.csv', {
            'parse_dates': ['date'],
            'dtype': {
                'player_id': 'object'
            }
        })
    ]
}

In [9]:
data = {}

In [10]:
# clubs
data['clubs'] = _data_['clubs'].copy()

data['clubs'] = data['clubs'][[
    'club_id', 
    'pretty_name'
]]
data['clubs'].rename(
    columns = {'pretty_name': 'club_name'},
    inplace = True
)
data['clubs'].set_index('club_id', inplace = True)

data['clubs']

,club_name
club_id,
1032,Fc Reading
2323,Orduspor
1387,Acn Siena 1904
3592,Kryvbas Kryvyi Rig
1071,Wigan Athletic
...,...
1269,Pec Zwolle
200,Fc Utrecht
317,Fc Twente Enschede


In [11]:
# games
data['games'] = _data_['games'].copy()

data['games'] = data['games'][[
    'season', 
    'game_id'
]]
data['games'].set_index('game_id', inplace = True)

data['games']

,season
game_id,
2244388,2012
2219794,2011
2244389,2012
2271112,2012
2229332,2012
...,...
3646190,2021
3646188,2021
3655616,2021


In [2]:
_data_['appearances'].dtypes

NameError: name '_data_' is not defined

In [13]:
# appearances
data['appearances'] = _data_['appearances'].copy()

data['appearances'] = data['appearances'].loc[
    :, ~data['appearances'].columns.isin([
        'appearance_id', 
        'competition_id'
    ])
]
data['appearances'].rename(
    columns = {'player_club_id': 'club_id'},
    inplace = True
)

data['appearances'] = (
    data['appearances']
        .merge(
            data['games'], 
            on = 'game_id',
            copy = False
        ).drop(columns = 'game_id')
        .merge(
            data['clubs'], 
            on = 'club_id',
            copy = False
        ).drop(columns = 'club_id')
)

data['appearances'] = (
    data['appearances']
        .groupby(['player_id', 'season'])
        .agg({
            **{
                c: 'sum' for c in [
                    'goals', 
                    'assists', 
                    'minutes_played', 
                    'yellow_cards', 
                    'red_cards'
                ]
            },
            'club_name': 'last'
        })
        .reset_index('season')
)

data['appearances']

,season,goals,assists,minutes_played,yellow_cards,red_cards,club_name
player_id,,,,,,,
10,2014,32,18,4578,12,0,Lazio Rom
10,2015,16,14,3428,6,0,Lazio Rom
100009,2014,0,0,5576,8,0,Kuban Krasnodar
100009,2015,2,2,4512,12,0,Kuban Krasnodar
100009,2016,0,0,1260,6,0,Anzhi Makhachkala
...,...,...,...,...,...,...,...
99923,2014,0,2,832,4,0,Cagliari Calcio
99924,2016,0,2,1824,6,0,Ca Osasuna
99977,2014,0,0,194,0,0,Rcd Mallorca


In [14]:
# player valuations
data['player_valuations'] = _data_['player_valuations'].copy()

data['player_valuations']['season'] = (
    pd.DatetimeIndex(data['player_valuations']['date']).year
)
data['player_valuations'].drop(columns = 'date', inplace = True)

data['player_valuations'] = (
    data['player_valuations']
        .groupby(['player_id', 'season'])
        .agg({'market_value': 'mean'})
        .reset_index('season')
)
data['player_valuations'].rename(
    columns = {'market_value_in_gbp': 'market_value'},
    inplace = True
)

data['player_valuations']

,season,market_value
player_id,,
10,2004,6300000.0
10,2005,10800000.0
10,2006,22500000.0
10,2007,20700000.0
10,2008,18000000.0
...,...,...
99977,2018,990000.0
99977,2019,720000.0
99977,2020,562500.0


In [15]:
# players
data['players'] = _data_['players'].copy()

data['players'] = data['players'].loc[
    :, ~data['players'].columns.isin([
        'last_season',
        'name',
        'current_club_id',
        'market_value_in_gbp',
        'highest_market_value_in_gbp',
        'country_of_birth',
        'url', 
        'foot'
    ])
]
data['players'].rename(
    columns = {
        'pretty_name': 'name',
        'height_in_cm': 'height',
        'country_of_citizenship': 'nationality'
    },
    inplace = True
)

data['players']['sub_position'] = (
    data['players']['sub_position'].cat
        .rename_categories(
            lambda s: (
                _re_replace(s, {
                    fr'''^(.*){_r(' - ')}(.*)$''': r'\2'
                })
                .title()
            )
        )
)

data['players'].set_index('player_id', inplace = True)

data['players']

,name,nationality,date_of_birth,position,sub_position,height
player_id,,,,,,
254016,Arthur Delalande,France,1992-05-18,Midfield,Central Midfield,186
51053,Daniel Davari,Iran,1988-01-06,Goalkeeper,Goalkeeper,192
31451,Torsten Oehrl,Germany,1986-01-07,Attack,Centre-Forward,192
44622,Vladimir Kisenkov,Russia,1981-10-08,Defender,Right-Back,182
30802,Oscar Diaz,Spain,1984-04-24,Attack,Centre-Forward,183
...,...,...,...,...,...,...
462285,Fabian De Keijzer,Netherlands,2000-05-10,Goalkeeper,Goalkeeper,193
368612,Merveille Bokadi,DR Congo,1996-05-21,Defender,Centre-Back,186
408574,Joey Veerman,Netherlands,1998-11-19,Midfield,Central Midfield,185


In [16]:
# final dataset
data['all'] = data['players'].merge(
    data['player_valuations'].merge(
        data['appearances'], 
        on = ['player_id', 'season'],
        copy = False
    ), 
    on = 'player_id',
    copy = False
)

data['all']['age'] = (
    pd.to_datetime(data['all']['season'], format = '%Y', utc = True) 
        - pd.to_datetime(data['all']['date_of_birth'], utc = True)
).astype('timedelta64[Y]')
data['all'].drop(columns = 'date_of_birth', inplace = True)

data['all'].dropna(axis = 'index', inplace = True)

data['all']

,name,nationality,position,sub_position,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,club_name,age
player_id,,,,,,,,,,,,,,
9800,Artem Milevskyi,Ukraine,Attack,Centre-Forward,189,2020,90000.0,0,0,720,6,0,Fk Minaj,34.0
43084,Gaetano Berardi,Switzerland,Defender,Right-Back,179,2020,360000.0,0,0,228,0,0,Leeds United,31.0
230826,Gennaro Acampora,Italy,Midfield,Central Midfield,174,2020,360000.0,2,4,1248,4,0,Spezia Calcio,25.0
198087,Matteo Ricci,Italy,Midfield,Defensive Midfield,176,2020,1530000.0,0,6,4880,10,0,Spezia Calcio,25.0
110689,Deniz Mehmet,Turkey,Goalkeeper,Goalkeeper,192,2020,68000.0,0,0,1080,0,0,Dundee United Fc,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364245,Jordan Teze,Netherlands,Defender,Centre-Back,183,2019,420000.0,0,0,360,0,0,Psv Eindhoven,19.0
364245,Jordan Teze,Netherlands,Defender,Centre-Back,183,2020,1102500.0,0,2,7494,10,0,Psv Eindhoven,20.0
364245,Jordan Teze,Netherlands,Defender,Centre-Back,183,2021,5400000.0,2,8,5260,12,0,Psv Eindhoven,21.0


## Data Evaluation

In [17]:
data['all'][data['all'].isna().any(axis = 1)]

,name,nationality,position,sub_position,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,club_name,age
player_id,,,,,,,,,,,,,,


In [18]:
data['all'].dtypes

name                object
nationality       category
position          category
sub_position      category
height               int64
season               int64
market_value       float64
goals                int64
assists              int64
minutes_played       int64
yellow_cards         int64
red_cards            int64
club_name           object
age                float64
dtype: object

In [19]:
data['all'].describe()

,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,age
count,50781.000000,50781.000000,5.078100e+04,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000
mean,180.794628,2017.380063,3.630890e+06,3.880546,2.949883,2805.795987,6.001575,0.143243,24.655284
std,17.703409,2.318805,8.274637e+06,7.352176,4.793814,2103.361660,6.095317,0.543827,4.375994
min,0.000000,2013.000000,9.000000e+03,0.000000,0.000000,2.000000,0.000000,0.000000,14.000000
25%,178.000000,2015.000000,3.600000e+05,0.000000,0.000000,884.000000,2.000000,0.000000,21.000000
50%,182.000000,2017.000000,9.000000e+05,0.000000,2.000000,2566.000000,4.000000,0.000000,24.000000
75%,187.000000,2019.000000,3.150000e+06,4.000000,4.000000,4410.000000,10.000000,0.000000,28.000000
max,206.000000,2021.000000,1.800000e+08,122.000000,62.000000,10122.000000,46.000000,6.000000,42.000000


In [20]:
pd.DataFrame(data['all']['sub_position'].unique())

,0
0,Centre-Forward
1,Right-Back
2,Central Midfield
3,Defensive Midfield
4,Goalkeeper
5,Centre-Back
6,Attacking Midfield
7,Right Winger
8,Left Winger
9,Left-Back


In [21]:
data['all'][data['all']['name'] == 'Cristiano Ronaldo']

,name,nationality,position,sub_position,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,club_name,age
player_id,,,,,,,,,,,,,,
8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,187,2014,96000000.0,122,46,9282,12,2,Real Madrid,28.0
8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,187,2015,105000000.0,102,30,8586,6,0,Real Madrid,29.0
8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,187,2016,99000000.0,84,24,8252,10,0,Real Madrid,30.0
8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,187,2017,90000000.0,88,16,7356,10,0,Real Madrid,31.0
8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,187,2018,96000000.0,56,20,7292,8,2,Juventus Turin,32.0
8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,187,2019,74250000.0,70,14,7982,6,0,Juventus Turin,33.0
8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,187,2020,54000000.0,76,8,7682,10,0,Juventus Turin,34.0
8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,187,2021,39000000.0,48,6,6202,20,0,Juventus Turin,35.0


## One hot encoding

In [22]:
# one hot encode categorical features
data['all_onehot'] = pd.get_dummies(data['all'], columns = [
    'position', 
    'sub_position', 
    'nationality', 
    'club_name'
])

data['all_onehot']

,name,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,age,...,club_name_West Bromwich Albion,club_name_West Ham United,club_name_Wigan Athletic,club_name_Willem Ii Tilburg,club_name_Wolverhampton Wanderers,club_name_Yeni Malatyaspor,club_name_Zenit St Petersburg,club_name_Zirka Kropyvnytskyi,club_name_Zorya Lugansk,club_name_Zska Moskau
player_id,,,,,,,,,,,,,,,,,,,,,
9800,Artem Milevskyi,189,2020,90000.0,0,0,720,6,0,34.0,...,0,0,0,0,0,0,0,0,0,0
43084,Gaetano Berardi,179,2020,360000.0,0,0,228,0,0,31.0,...,0,0,0,0,0,0,0,0,0,0
230826,Gennaro Acampora,174,2020,360000.0,2,4,1248,4,0,25.0,...,0,0,0,0,0,0,0,0,0,0
198087,Matteo Ricci,176,2020,1530000.0,0,6,4880,10,0,25.0,...,0,0,0,0,0,0,0,0,0,0
110689,Deniz Mehmet,192,2020,68000.0,0,0,1080,0,0,27.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364245,Jordan Teze,183,2019,420000.0,0,0,360,0,0,19.0,...,0,0,0,0,0,0,0,0,0,0
364245,Jordan Teze,183,2020,1102500.0,0,2,7494,10,0,20.0,...,0,0,0,0,0,0,0,0,0,0
364245,Jordan Teze,183,2021,5400000.0,2,8,5260,12,0,21.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
data['all_onehot'].dtypes

name                              object
height                             int64
season                             int64
market_value                     float64
goals                              int64
                                  ...   
club_name_Yeni Malatyaspor         uint8
club_name_Zenit St Petersburg      uint8
club_name_Zirka Kropyvnytskyi      uint8
club_name_Zorya Lugansk            uint8
club_name_Zska Moskau              uint8
Length: 588, dtype: object

In [24]:
data['all_onehot'].describe()

,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,age,position_Attack,...,club_name_West Bromwich Albion,club_name_West Ham United,club_name_Wigan Athletic,club_name_Willem Ii Tilburg,club_name_Wolverhampton Wanderers,club_name_Yeni Malatyaspor,club_name_Zenit St Petersburg,club_name_Zirka Kropyvnytskyi,club_name_Zorya Lugansk,club_name_Zska Moskau
count,50781.000000,50781.000000,5.078100e+04,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,...,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000,50781.000000
mean,180.794628,2017.380063,3.630890e+06,3.880546,2.949883,2805.795987,6.001575,0.143243,24.655284,0.339674,...,0.002304,0.003604,0.000118,0.003564,0.002028,0.002560,0.003663,0.001339,0.003998,0.003938
std,17.703409,2.318805,8.274637e+06,7.352176,4.793814,2103.361660,6.095317,0.543827,4.375994,0.473603,...,0.047945,0.059923,0.010869,0.059596,0.044992,0.050532,0.060411,0.036569,0.063100,0.062634
min,0.000000,2013.000000,9.000000e+03,0.000000,0.000000,2.000000,0.000000,0.000000,14.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,178.000000,2015.000000,3.600000e+05,0.000000,0.000000,884.000000,2.000000,0.000000,21.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,182.000000,2017.000000,9.000000e+05,0.000000,2.000000,2566.000000,4.000000,0.000000,24.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,187.000000,2019.000000,3.150000e+06,4.000000,4.000000,4410.000000,10.000000,0.000000,28.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,206.000000,2021.000000,1.800000e+08,122.000000,62.000000,10122.000000,46.000000,6.000000,42.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
data['all_onehot'][data['all_onehot']['name'] == 'Lionel Messi']

,name,height,season,market_value,goals,assists,minutes_played,yellow_cards,red_cards,age,...,club_name_West Bromwich Albion,club_name_West Ham United,club_name_Wigan Athletic,club_name_Willem Ii Tilburg,club_name_Wolverhampton Wanderers,club_name_Yeni Malatyaspor,club_name_Zenit St Petersburg,club_name_Zirka Kropyvnytskyi,club_name_Zorya Lugansk,club_name_Zska Moskau
player_id,,,,,,,,,,,,,,,,,,,,,
28003,Lionel Messi,169,2014,108000000.0,116,62,10122,12,0,26.0,...,0,0,0,0,0,0,0,0,0,0
28003,Lionel Messi,169,2015,108000000.0,82,48,8458,10,0,27.0,...,0,0,0,0,0,0,0,0,0,0
28003,Lionel Messi,169,2016,108000000.0,108,40,8904,18,0,28.0,...,0,0,0,0,0,0,0,0,0,0
28003,Lionel Messi,169,2017,108000000.0,90,40,8936,14,0,29.0,...,0,0,0,0,0,0,0,0,0,0
28003,Lionel Messi,169,2018,156000000.0,102,44,8048,6,0,30.0,...,0,0,0,0,0,0,0,0,0,0
28003,Lionel Messi,169,2019,130500000.0,60,50,7262,14,0,31.0,...,0,0,0,0,0,0,0,0,0,0
28003,Lionel Messi,169,2020,95400000.0,78,30,8746,12,2,32.0,...,0,0,0,0,0,0,0,0,0,0
28003,Lionel Messi,169,2021,66000000.0,22,26,5384,2,0,33.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

corr_matrix = df.corr()

data['all_eda']['log_market_value'] = np.log(data['all_eda']['market_value'])

corr = data['all_eda'].corr()
_ = sns.heatmap(corr_matrix,  
    cmap = 'seismic', 
    linewidth = 1, linecolor = 'white',
    vmax = 1, vmin = -1,
    mask = np.triu(np.ones_like(corr, dtype = bool)), 
    annot = True,
    fmt = '0.2f'
)

In [ ]:
sns.pairplot(data['all'])

# Evaluation Metrics

We will be using an OLS regression model and the evaluation techniques we are considering are RMSE and Euclidean distance.
A possible evaluation metric we will use is RMSE or Mean Absolute Value of Errors. It is derived by calculating the difference between the estimated and actual value, square those results, then calculate the mean of those results. The formula for RMSE is 

$$
    \text{RMSE} = \sqrt{\frac{\sum_{i = 1}^{N}\left(\text{Predicted}_{i} - \text{Actual}_{i}\right)^{2}}{N}}
$$

## Preliminary results

In [ ]:
# feature selection
!{sys.executable} -m pip install --quiet patsy

import patsy
y, X = patsy.dmatrices("market_value ~ age +  goals + assists + minutes_played +yellow_cards +red_cards +height +age", data=data['all'], return_type="dataframe")
model = sm.OLS(y, X)
fit = model.fit()
pred = fit.predict(X)
fit.summary()

In [ ]:
print((y))
print((pred))

np.sqrt(mean_squared_error((y),(pred)))

In [ ]:
data['all']['log_market_value'] = np.log(data['all']['market_value'])
y, X = patsy.dmatrices("log_market_value ~ age +  goals + assists + minutes_played +yellow_cards +red_cards +height +age ", data=data['all'], return_type="dataframe")
model = sm.OLS(y, X)
fit = model.fit()
pred2 = fit.predict(X)
fit.summary()

In [ ]:
print(np.exp(y))
print(np.exp(pred2))

np.sqrt(mean_squared_error((y),(pred2)))

In [ ]:
y, X = patsy.dmatrices("log_market_value ~ age +  goals + assists + minutes_played +yellow_cards +red_cards +height +age + C(nationality)+ C(position)+ C(sub_position) +C(club_name)", data=data['all'], return_type="dataframe")
model = sm.OLS(y, X)
fit = model.fit()
pred3 = fit.predict(X)
fit.summary()

In [ ]:
print(np.exp(y))
print(np.exp(pred3))

np.sqrt(mean_squared_error((y),(pred3)))

In [ ]:
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
def diagnostic_plot(x, y):
    plt.figure(figsize=(20,5))
    
    rgr = LinearRegression()
    rgr.fit(x,y)
    pred = rgr.predict(x)

    plt.subplot(1, 3, 1)
    plt.scatter(pred,y,alpha=0.1)
    plt.plot(y, y, color='red',linewidth=1,)
    plt.title("Regression fit")
    plt.xlabel("Predicted y")
    plt.ylabel("y")
    
y = data['all']['log_market_value']
X = data['all'][[
    'goals', 
    'assists', 
    'minutes_played', 
    'yellow_cards', 
    'red_cards', 
    'height',
    'age'
   # 'nationality',
   # 'position',
   # 'sub_position',
   # 'club_name'
]]

In [ ]:
sns.set(style='darkgrid')
diagnostic_plot(X, y)


In [ ]:
y = data['all']['market_value']
dataset = pd.get_dummies(data['all'], columns = [
    'position', 
    'sub_position', 
    'nationality', 
    'club_name'
])

dataset = dataset.drop(columns = ['name','market_value'])
#dataset

X_train, X_test, Y_train, Y_test = train_test_split(dataset, y, test_size = .30, random_state = 70)
    
regr = linear_model.LinearRegression() # Do not use fit_intercept = False if you have removed 1 column after dummy encoding
regr.fit(X_train, Y_train)

predicted = regr.predict(X_test)
#predicted2 = regr.predict(X_train[:1])

print(regr.score(X_train, Y_train))

print(regr.score(X_test, Y_test))

In [ ]:
sns.displot(data = data['all'], x = 'market_value', bins = 50)

plt.show()

In [ ]:
df_highest_market_value_players = data['all'].nlargest(n = 1, columns = 'market_value')

df_highest_market_value_players

In [ ]:
df_highest_market_value = data['all'].loc[data['all']['name'].isin(df_highest_market_value_players['name'])]

df_highest_market_value

In [ ]:
sns.barplot(data = df_highest_market_value, x = 'season', y = 'market_value')
plt.title('Player Market Value Over the Years')
plt.xticks(rotation = 45, ha = 'right', rotation_mode = 'anchor')

plt.show()

In [ ]:
y = data['all']['market_value']
dataset = pd.get_dummies(data['all'], columns = [
    'position', 
    'sub_position', 
    'nationality', 
    'club_name'
])

dataset = dataset.drop(columns = ['name','market_value'])
dataset

In [ ]:
# from sklearn import linear_model
# from sklearn.compose import ColumnTransformer
# from sklearn.datasets import fetch_openml
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import ElasticNet
# from sklearn.model_selection import KFold

# '''
# regr = Pipeline(
#     #steps=[("preprocessor", preprocessor), ("model", Lasso())]
#     steps=[("preprocessor", preprocessor), ("model", gscv )]
# )
# gscv = GridSearchCV(regr, param_grid={'alpha': [0.01, 0.5, 1]}, cv=KFold(5))
# '''

# X_train, X_test, Y_train, Y_test = train_test_split(dataset, y, test_size = .30, random_state = 70)
    
# regr = linear_model.LinearRegression() # Do not use fit_intercept = False if you have removed 1 column after dummy encoding
# regr.fit(X_train, Y_train)

# predicted = regr.predict(X_test)
# #predicted2 = regr.predict(X_train[:1])

# print(regr.score(X_train, Y_train))



# print(regr.score(X_test, Y_test))


In [ ]:
# print(predicted)
# print(Y_test)


In [ ]:
y = data['all']['log_market_value']
X = data['all'][[
    'goals', 
    'assists', 
    'minutes_played', 
    'yellow_cards', 
    'red_cards', 
    'height',
    'age',
    'nationality',
    'position',
    'sub_position',
    'club_name'
]]
print(data['all'])

In [ ]:
# move these to the top 
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold

num_features = ['goals','assists','minutes_played', 'yellow_cards', 'red_cards', 'height','age']
num_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)
categorical_features = ['nationality', 'position', 'sub_position', 'club_name']
cat_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_features),
        ("cat", cat_transformer, categorical_features),
    ]
)


In [ ]:
clf = Pipeline(
    #steps=[("preprocessor", preprocessor), ("model", Lasso())]
    steps=[("preprocessor", preprocessor), ("model", LinearRegression())]
)
#gscv = GridSearchCV(clf, param_grid={'alpha': [0.01, 0.5, 1]}, cv=KFold(5))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
clf.fit(X_train,y_train)
score = clf.score(X_test,y_test)
y_pred = clf.predict(X_test)
print(score)
print(y_pred[:5])
print(y_test[:5])

In [ ]:
# heat map
corr_matrix = data['all'].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
_ = sns.heatmap(corr_matrix,cmap='seismic',linewidth=1,linecolor='white',vmax = 1, vmin=-1,mask=mask, annot=True,fmt='0.2f')

## Footnotes

<sup><a name="cariboo" href="#note-cariboo">7</a></sup> *Football Data from Transfermarkt.* (2022, April 22). [Dataset]. https://www.kaggle.com/datasets/davidcariboo/player-scores